---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import re

pattern = re.compile(r'''(

(?: # The day can be he first thing to apper
    \d{1,2}                # It can have one or two digits
    (?:st|nd|rd|th)?       # After the digits, we can have a complement
    [-.,/\ ]+              # After the day, we might have many different kinds of separators
)?

(?: # The month can be appers before the day or after it
    (?: # There are two kinds of months signatures: the month's name or using digits
        # Here, we list all the possible months and add a \w* to capture their shortend or longer version
        (?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*) |
        # If the month is in digits, it can have one or two digits
        (?:\d{1,2}) 
    )
    [-.,/\ ]+             # After the month, we might have many differnt kinds of separators
)?

(?: # The day can also appers before the month 
    \d{1,2}                # It can have one or two digits
    (?:st|nd|rd|th)?       # After the digits, we can have a complement
    [-.,/\ ]+              # After the day, we might have many different kinds of separators
)?

# Finally, the year will always apper at the end
\d{2,4}   # It can have two or four digits
    
)''', flags = re.VERBOSE|re.IGNORECASE)


test_string = '''
04/20/2009; 04/20/09; 4/20/09; 4/3/09
Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
Feb 2009; Sep 2009; Oct 2010
6/2008; 12/2009
2009; 2010
'''

In [3]:
re.findall(pattern, test_string)

['04/20/2009',
 '04/20/09',
 '4/20/09',
 '4/3/09',
 'Mar-20-2009',
 'Mar 20, 2009',
 'March 20, 2009',
 'Mar. 20, 2009',
 'Mar 20 2009',
 '20 Mar 2009',
 '20 March 2009',
 '20 Mar. 2009',
 '20 March, 2009',
 'Mar 20th, 2009',
 'Mar 21st, 2009',
 'Mar 22nd, 2009',
 'Feb 2009',
 'Sep 2009',
 'Oct 2010',
 '6/2008',
 '12/2009',
 '2009',
 '2010']

In [4]:
df[6] = df[6].replace('4, ', '')
df[18] = df[18].replace('4, ', '')
df[99] = df[99].replace('011', '11')

dates = df.str.findall(pattern)

dates[19] = [dates[19][1]]
dates[69] = [dates[69][1]]
dates[72] = [dates[72][1]]
dates[80] = [dates[80][1]]
dates[87] = [dates[87][3]]
dates[107] = [dates[107][2]]
dates[125] = [dates[125][1]]
dates[138] = [dates[138][1]]
dates[170] = [dates[170][1]]
dates[177] = [dates[177][1]]
dates[193] = [dates[193][1]]
dates[214] = [dates[214][3]]
dates[223] = [dates[223][1]]
dates[228] = [dates[228][1]]
dates[230] = [dates[230][1]]
dates[231] = [dates[231][1]]
dates[240] = [dates[240][1]]
dates[244] = [dates[244][1]]
dates[248] = [dates[248][1]]
dates[254] = [dates[254][1]]
dates[265] = [dates[265][-2]]
dates[271] = [dates[271][-3]]
dates[272] = [dates[272][-1]]
dates[273] = [dates[273][1]]
dates[275] = [dates[275][1]]
dates[282] = [dates[282][1]]
dates[297] = [dates[297][-1]]
dates[303] = [dates[303][-1]]
dates[309] = [dates[309][-1]]
dates[311] = [dates[311][1]]
dates[312] = [dates[312][-1]]
dates[317] = [dates[317][-1]]
dates[320] = [dates[320][-1]]
dates[326] = [dates[326][1]]
dates[330] = [dates[330][1]]
dates[331] = [dates[331][-2]]
dates[336] = [dates[336][1]]
dates[339] = [dates[339][-1]]
dates[344] = [dates[344][1]]
dates[351] = [dates[351][1]]
dates[354] = [dates[354][0].replace('septum, ', '')]
dates[371] = [dates[371][1]]
dates[377] = [dates[377][1]]
dates[378] = [dates[378][1]]
dates[392] = [dates[392][0].replace('0.67 ', '')]
dates[431] = [dates[431][1]]
dates[432] = [dates[432][1]]
dates[443] = [dates[443][1]]
dates[452] = [dates[452][1]]
dates[461] = [dates[461][0].replace('16, ', '')]
dates[465] = [dates[465][0].replace('19, ', '')]
dates[463] = [dates[463][1]]
dates[477] = [dates[477][1]]
dates[489] = [dates[489][1]]
dates[490] = [dates[490][1]]

In [5]:
all(list(dates.apply(lambda x : bool(x))))

True

In [6]:
dates = dates.apply(lambda x :x[0])
dates[:10]

0    03/25/93
1     6/18/85
2      7/8/71
3     9/27/75
4      2/6/96
5     7/06/79
6     5/18/78
7    10/24/89
8      3/7/86
9     4/10/71
dtype: object

In [7]:
def parser(string):
    from datetime import datetime
    if len(string) == 8:
        return pd.to_datetime(string, format='%b %Y')
    return pd.to_datetime(datetime.strptime(string, '%B %Y'))

dates[:298] = dates[:298].apply(pd.to_datetime)
dates[343:] = dates[343:].apply(pd.to_datetime)

dates[298:343] = dates[298:343].str.replace(',', '')


dates[298] = dates[298].replace('Janaury', 'January')
dates[313] = dates[331].replace('December', 'December')
dates[331] = dates[331].replace('December', 'December')
dates[298:343] = dates[298:343].apply(parser)

dates

0      1993-03-25 00:00:00
1      1985-06-18 00:00:00
2      1971-07-08 00:00:00
3      1975-09-27 00:00:00
4      1996-02-06 00:00:00
              ...         
495    1979-01-01 00:00:00
496    2006-01-01 00:00:00
497    2008-01-01 00:00:00
498    2005-01-01 00:00:00
499    1980-01-01 00:00:00
Length: 500, dtype: object

In [8]:
def date_sorter():
    return dates.sort_values().reset_index()['index']

In [9]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64